# Tutorial


This tutorial runs you through the process of running inferences for a deployments in Costa Rica.

In [ ]:
import os
os.chdir('/Users/kgoldmann/Documents/Projects/AMBER/amber-inferences')
os.system('pwd')

In [ ]:
# Install the package if required
!pip install -e .

In [ ]:
import amber_inferences
import boto3
import pandas as pd
from amber_inferences.utils.config import load_credentials
from amber_inferences.utils.api_utils import get_buckets, deployments_summary, get_deployments

# Explore the Data on the Object Store

In [ ]:
# Create an instance for the object store
aws_credentials = load_credentials('./credentials.json')
session = boto3.Session(
        aws_access_key_id=aws_credentials["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=aws_credentials["AWS_SECRET_ACCESS_KEY"],
        region_name=aws_credentials["AWS_REGION"],
    )
s3_client = session.client("s3", endpoint_url=aws_credentials["AWS_URL_ENDPOINT"])

Look at the deployments available on the object store:

In [ ]:
all_deployments = get_deployments(aws_credentials['UKCEH_username'], aws_credentials['UKCEH_password'])

all_deployments = pd.DataFrame(all_deployments)

List the buckets/countries:

In [ ]:
all_deployments[all_deployments['status'] == 'active']

Let's pick one, cri (Costa Rica) and check out the data attached. 

In [ ]:
cr_deployments = deployments_summary(
    aws_credentials,
    subset_countries=["Costa Rica"],
    subset_deployments=["dep000031", "dep000032"],
    include_image_count=False
)

To get the files for a given deployment(s):

In [ ]:
cr_deployments

# Log the image keys

In [ ]:
from amber_inferences.utils.key_utils import save_keys

In [ ]:
# takes some time to commenting out to save time
# save_keys(
#     s3_client,
#     bucket="cri",
#     deployment_id="dep000031",
#     output_file="./examples/dep000031_keys.json",
#     subdir="snapshot_images"
# )

In [ ]:
# Look at the keys
!head ./examples/dep000031_keys.json

# Download the images

In [ ]:
import json
from amber_inferences.utils.inference_scripts import download_image_from_key

In [ ]:
# read the first image in the keys file and open
with open('./examples/dep000031_keys.json') as f:
    keys = json.load(f)

In [ ]:
for i in range(100, 110):
    download_image_from_key(s3_client, keys[i], 'cri', './examples/images/dep000031/')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
image_paths = os.listdir('./examples/images/dep000031/')
image_paths = [os.path.join('./examples/images/dep000031/', x) for x in image_paths]

In [ ]:
# Open the images in a 2x5 grid
fig, axs = plt.subplots(2, 5, figsize=(20, 10))
axs = axs.ravel()

for i, img_path in enumerate(image_paths):
    if os.path.exists( img_path):  # Ensure the file exists
        img = mpimg.imread(img_path)
        axs[i].imshow(img)
        axs[i].axis("off")  # Hide axes for better visualization
        axs[i].set_title(f"Image {i+1}")
    else:
        axs[i].axis("off")
        axs[i].set_title("Missing Image")

plt.tight_layout()
plt.show()

# Perform Object Detection on the Images

In [ ]:
from amber_inferences.utils.custom_models import *
from amber_inferences.utils.inference_scripts import *
import torch

In [ ]:
models = load_models(
    device=torch.device("cpu"),
    localisation_model_path='./models/v1_localizmodel_2021-08-17-12-06.pt',
    binary_model_path='./models/moth-nonmoth-effv2b3_20220506_061527_30.pth',
    order_model_path='./models/dhc_best_128.pth',
    order_threshold_path='./models/thresholdsTestTrain.csv',
    species_model_path='./models/turing-costarica_v03_resnet50_2024-06-04-16-17_state.pt',
    species_labels='./models/03_costarica_data_category_map.json'
)

In [ ]:
crops = crop_image_only(
    image_path='./examples/images/dep000031/01-20240131213830-snapshot.jpg',
    bucket_name="cri",
    localisation_model=models['localisation_model'],
    proc_device=torch.device("cpu"),
    csv_file="./examples/dep000031_crops.csv",
    save_crops=True,
    box_threshold=0.99,
    crop_dir="./examples/crops/dep000031",
    job_name=None,
)

In [ ]:
crops

annotate the input image with the bounding boxes

In [ ]:
# annotate the image with the bounding boxes
from PIL import Image
from PIL import ImageDraw

def image_annotation(image_path, img=None, boxes={}):
    if img is None:
        img = Image.open(image_path)

    draw = ImageDraw.Draw(img)
    for box in boxes:
        x0 = box['x_min']
        y0 = box['y_min']
        x1 = box['x_max']
        y1 = box['y_max']
        draw.rectangle([x0, y0, x1, y1], outline='red', width=3)

        if 'label' in box.keys():
            draw.text((x0, y0), box['label'], fill='red')

    plt.imshow(img)



In [ ]:
image_path = '/Users/kgoldmann/Documents/Projects/AMBER/amber-inferences/examples/images/dep000031/01-20240131213830-snapshot.jpg'
crops_df = crops.loc[crops['image_path'] == image_path]

# create a dict of the bounding boxes
boxes = []
for i, row in crops_df.iterrows():
    boxes.append({
        'x_min': row['x_min'],
        'y_min': row['y_min'],
        'x_max': row['x_max'],
        'y_max': row['y_max']
    })

In [ ]:
image_annotation(image_path, boxes=boxes)

# Binary Classifier

In [ ]:
# Extract the crop
transform_species = transforms.Compose(
        [
            transforms.Resize((300, 300)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ]
    )

image = Image.open(image_path).convert("RGB")
original_image = image.copy()
original_width, original_height = image.size

crops_df = crops.loc[crops['image_path'] == image_path]
x_min, y_min, x_max, y_max = crops_df.iloc[0][['x_min', 'y_min', 'x_max', 'y_max']]

cropped_image = original_image.crop((x_min, y_min, x_max, y_max))
cropped_tensor = transform_species(cropped_image).unsqueeze(0).to(torch.device('cpu'))

In [ ]:
binary_prediction = classify_box(cropped_tensor, models['classification_model'])

print(f'Crop predicted to be {binary_prediction[0]} with {"{:.2f}".format(binary_prediction[1]*100)}% accuracy')

# Order Classifier

In [ ]:
order_prediction = classify_order(
    cropped_tensor,
    models['order_model'],
    models['order_model_labels'],
    models['order_model_thresholds']
)

print(f'Crop predicted to be {order_prediction[0]} with {"{:.2f}".format(order_prediction[1]*100)}% accuracy')

# Species Classifier

In [ ]:
species_names, species_confidences = classify_species(
    cropped_tensor,
    models['species_model'],
    models['species_model_labels'],
    5
)

In [ ]:
prediction_df = pd.DataFrame({'species':species_names, 'confidence':species_confidences})

prediction_df

## Looking at the predictions

In [ ]:
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from amber_inferences.utils.image_processing import *

In [ ]:
def display_images_grid(local_image, prediction_df):
    """Displays a 2x3 grid with a local image + 5 images from GBIF."""

    # Create 2x3 plot grid
    fig, axes = plt.subplots(2, 3, figsize=(10, 7))

    # Display local image in top-left
    axes[0, 0].imshow(local_image)
    axes[0, 0].set_title("Crop Image")
    axes[0, 0].axis("off")

    # Fetch and display GBIF images
    for i, sp in enumerate(prediction_df['species']):
        row, col = divmod(i + 1, 3)  # Skip first slot (0,0) for crop image
        img = get_gbif_image(sp)

        if img:
            axes[row, col].imshow(img)
        else:
            axes[row, col].text(0.5, 0.5, "No Image", ha="center", va="center")

        axes[row, col].set_title(f"{sp}, Liklihood: {prediction_df['confidence'][i]:.2f}")
        axes[row, col].axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage
display_images_grid(cropped_image, prediction_df)